# Decorators
## What are decorators?
"Decoration is a way to specify management code for functions and classes." ... "A decorator itself is a callable that returns a callable." - Mark Lutz

A decorator wraps a function without modifying the function itself. The result of the wrapping?
* Adds functionality to the function.
* Modifies the behavior of the function.

A simple example probably illustrates this well:

In [ ]:
def myDecorator(func):
    def new_func(n):
        return '$' + str(func(n))        
    return new_func

In [ ]:
def myFunction(a):
    return(a)

In [ ]:
print(myFunction(100))
type(myFunction(100))

In [ ]:
result = myDecorator(myFunction)(100)
print(result)
type(result)

This is nice, but there is shorthand for this approach

In [ ]:
@myDecorator
def myFunction(a):
    return(a)

In [ ]:
# call the decorated function
print(myFunction(100))
type(myFunction(100))

## Our use case: tasks that leverage Dask
In our use case (MPI/OpenMP-enabled tasks) we want to use Dask as a task manager. This requires a lot of tweaking based on the machine resources (how many OpenMP threads, how many nodes for a given `ncores`, what additional environment variables are required,...). The necessary logic for this can all be hidden away, and decorators are a great way for us to do this while ensuring what we do is compatible with Dask.

In [ ]:
from dask.distributed import LocalCluster

In [ ]:
from jobqueue_features.decorators import on_cluster, task
from jobqueue_features.functions import set_default_cluster
from jobqueue_features.clusters import CustomSLURMCluster

In [ ]:
set_default_cluster(LocalCluster)

In [ ]:
@task()
def square(x):
    return x ** 2

In [ ]:
@on_cluster()
def simple_taskset():
    sq_tasks = list(map(square, range(1, 11)))
    print([t.result() for t in sq_tasks])

In [ ]:
%%time
simple_taskset()

To look inside what is happening, we can turn on logging

In [ ]:
import logging

logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.DEBUG)

But for the logged case, rather than use the default let's do a more interesting example

In [ ]:
custom_cluster = CustomSLURMCluster(
    name="myCluster"
)

In [ ]:
custom_cluster

In [ ]:
@task(cluster=custom_cluster)
def square(x):
    return x ** 2

In [ ]:
@on_cluster(cluster=custom_cluster)
def simple_taskset():
    sq_tasks = list(map(square, range(1, 11)))
    print([t.result() for t in sq_tasks])

In [ ]:
%%time
simple_taskset()